# 1、基础使用

举例

In [15]:
# 1.导入相关依赖
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv

dotenv.load_dotenv()

# 2.定义文档加载器
loader = TextLoader(file_path='./asset/load/09-ai1.txt', encoding="utf-8")

# 3.加载文档
documents = loader.load()

# 4.定义文本切割器
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 5.切割文档
docs = text_splitter.split_documents(documents)

# 6.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [22]:
from langchain_community.vectorstores import FAISS

print("type(docs[0]) ==>",type(docs[0]))
print("type(docs) ==>",type(docs))
print("type(embeddings) ==>",type(embeddings))

# 7.将文档存储到向量数据库中
db = FAISS.from_documents(docs, embeddings)

# 8.从向量数据库中得到检索器
retriever = db.as_retriever()

# 9.使用检索器检索
docs = retriever.invoke("深度学习是什么？")

print(len(docs))

# 10.得到结果
for doc in docs:
    print(f"⭐{doc}")

type(docs[0]) ==> <class 'langchain_core.documents.base.Document'>
type(docs) ==> <class 'list'>
type(embeddings) ==> <class 'langchain_openai.embeddings.base.OpenAIEmbeddings'>


ValueError: input not a numpy array